# Day 13: Distress Signal

## Part 1

You climb the hill and again try contacting the Elves. However, you instead receive a signal you weren't expecting: a distress signal.

Your handheld device must still not be working properly; the packets from the distress signal got decoded out of order. You'll need to re-order the list of received packets (your puzzle input) to decode the message.

Your list consists of pairs of packets; pairs are separated by a blank line. You need to identify how many pairs of packets are in the right order.

For example:
```
[1,1,3,1,1]
[1,1,5,1,1]

[[1],[2,3,4]]
[[1],4]

[9]
[[8,7,6]]

[[4,4],4,4]
[[4,4],4,4,4]

[7,7,7,7]
[7,7,7]

[]
[3]

[[[]]]
[[]]

[1,[2,[3,[4,[5,6,7]]]],8,9]
[1,[2,[3,[4,[5,6,0]]]],8,9]
```

Packet data consists of lists and integers. Each list starts with [, ends with ], and contains zero or more comma-separated values (either integers or other lists). Each packet is always a list and appears on its own line.

When comparing two values, the first value is called left and the second value is called right. Then:

- If both values are integers, the lower integer should come first. If the left integer is lower than the right integer, the inputs are in the right order. If the left integer is higher than the right integer, the inputs are not in the right order. Otherwise, the inputs are the same integer; continue checking the next part of the input.
- If both values are lists, compare the first value of each list, then the second value, and so on. If the left list runs out of items first, the inputs are in the right order. If the right list runs out of items first, the inputs are not in the right order. If the lists are the same length and no comparison makes a decision about the order, continue checking the next part of the input.
- If exactly one value is an integer, convert the integer to a list which contains that integer as its only value, then retry the comparison. For example, if comparing [0,0,0] and 2, convert the right value to [2] (a list containing 2); the result is then found by instead comparing [0,0,0] and [2].


Using these rules, you can determine which of the pairs in the example are in the right order:

```
== Pair 1 ==
- Compare [1,1,3,1,1] vs [1,1,5,1,1]
  - Compare 1 vs 1
  - Compare 1 vs 1
  - Compare 3 vs 5
    - Left side is smaller, so inputs are in the right order

== Pair 2 ==
- Compare [[1],[2,3,4]] vs [[1],4]
  - Compare [1] vs [1]
    - Compare 1 vs 1
  - Compare [2,3,4] vs 4
    - Mixed types; convert right to [4] and retry comparison
    - Compare [2,3,4] vs [4]
      - Compare 2 vs 4
        - Left side is smaller, so inputs are in the right order

== Pair 3 ==
- Compare [9] vs [[8,7,6]]
  - Compare 9 vs [8,7,6]
    - Mixed types; convert left to [9] and retry comparison
    - Compare [9] vs [8,7,6]
      - Compare 9 vs 8
        - Right side is smaller, so inputs are not in the right order

== Pair 4 ==
- Compare [[4,4],4,4] vs [[4,4],4,4,4]
  - Compare [4,4] vs [4,4]
    - Compare 4 vs 4
    - Compare 4 vs 4
  - Compare 4 vs 4
  - Compare 4 vs 4
  - Left side ran out of items, so inputs are in the right order

== Pair 5 ==
- Compare [7,7,7,7] vs [7,7,7]
  - Compare 7 vs 7
  - Compare 7 vs 7
  - Compare 7 vs 7
  - Right side ran out of items, so inputs are not in the right order

== Pair 6 ==
- Compare [] vs [3]
  - Left side ran out of items, so inputs are in the right order

== Pair 7 ==
- Compare [[[]]] vs [[]]
  - Compare [[]] vs []
    - Right side ran out of items, so inputs are not in the right order

== Pair 8 ==
- Compare [1,[2,[3,[4,[5,6,7]]]],8,9] vs [1,[2,[3,[4,[5,6,0]]]],8,9]
  - Compare 1 vs 1
  - Compare [2,[3,[4,[5,6,7]]]] vs [2,[3,[4,[5,6,0]]]]
    - Compare 2 vs 2
    - Compare [3,[4,[5,6,7]]] vs [3,[4,[5,6,0]]]
      - Compare 3 vs 3
      - Compare [4,[5,6,7]] vs [4,[5,6,0]]
        - Compare 4 vs 4
        - Compare [5,6,7] vs [5,6,0]
          - Compare 5 vs 5
          - Compare 6 vs 6
          - Compare 7 vs 0
            - Right side is smaller, so inputs are not in the right order
```

What are the indices of the pairs that are already in the right order? (The first pair has index 1, the second pair has index 2, and so on.) In the above example, the pairs in the right order are 1, 2, 4, and 6; the sum of these indices is 13.

Determine which pairs of packets are already in the right order. What is the sum of the indices of those pairs?



In [18]:
import ast

# Loading in the input file
with open('input.txt') as f:
    distress_pairs = f.read().splitlines()

dummy_distress_pairs = [
    '[1,1,3,1,1]',
    '[1,1,5,1,1]',
    '',
    '[[1],[2,3,4]]',
    '[[1],4]',
    '',
    '[9]',
    '[[8,7,6]]',
    '',
    '[[4,4],4,4]',
    '[[4,4],4,4,4]',
    '',
    '[7,7,7,7]',
    '[7,7,7]',
    '',
    '[]',
    '[3]',
    '',
    '[[[]]]',
    '[[]]',
    '',
    '[1,[2,[3,[4,[5,6,7]]]],8,9]',
    '[1,[2,[3,[4,[5,6,0]]]],8,9]',
    ''
]

# distress_pairs = dummy_distress_pairs

In [19]:
# Instantiating placeholders to hold batches
distress_batches = []
current_batch = []

for entry in distress_pairs:
    
    # Adding batch pairs if line break occurs
    if entry == '':
        distress_batches.append(current_batch)
        current_batch = []
    
    # Converting list as string to acual list and adding it to current batch
    else:
        actual_list = ast.literal_eval(entry)
        current_batch.append(actual_list)

In [20]:
def compare_items(left, right):
    '''
    Comparing the items between the left and right batchs
    
    Inputs:
        - left (list or int): The left batch or item
        - right (list or int): The right batch or item

    Returns:
        - (sort_value) (int): A value that represents the correct action
    '''

    # Converting the types into lists if either left or right is currently a list
    if isinstance(left, list) and isinstance(right, int):
        right = [right]
    elif isinstance(left, int) and isinstance(right, list):
        left = [left]

    # Returning the appropriate value of both inputs are integers
    if isinstance(left, int) and isinstance(right, int):
        if left < right:
            return 1
        elif left == right:
            return 0
        else:
            return -1

    # Recursively comparing items if both inputs are lists
    if isinstance(left, list) and isinstance(right, list):

        # Instantiating an index value to iterate over both lists
        i = 0

        # Continuing to loop over the lists while there are still items
        while i < len(left) and i < len(right):

            # Recursively comparing the items again
            recursive_result = compare_items(left[i], right[i])

            # Returning the appropriate results
            if recursive_result == 1:
                return 1
            elif recursive_result == -1:
                return -1
            
            i += 1
        
        # Checking to see if there are any items remaining in the right input if the left input completes
        if i == len(left):
            if len(right) == len(left):
                return 0
            else:
                return 1
            
        return -1


In [21]:
# Initializing a value to represent the sum of the indices
sum_indices = 0

# Iterating over all the batch pairs
for i, batch in enumerate(distress_batches):
    
    # Extracting the left and right batches
    left_batch = batch[0]
    right_batch = batch[1]

    # Incrementing any results where the output of the comparison function is 1
    if compare_items(left_batch, right_batch) == 1:
        sum_indices += i + 1

print(f'Final result: {sum_indices}')

Final result: 6187


## Part 2

Now, you just need to put all of the packets in the right order. Disregard the blank lines in your list of received packets.

The distress signal protocol also requires that you include two additional divider packets:
```
[[2]]
[[6]]
``````
Using the same rules as before, organize all packets - the ones in your list of received packets as well as the two divider packets - into the correct order.

For the example above, the result of putting the packets in the correct order is:
```
[]
[[]]
[[[]]]
[1,1,3,1,1]
[1,1,5,1,1]
[[1],[2,3,4]]
[1,[2,[3,[4,[5,6,0]]]],8,9]
[1,[2,[3,[4,[5,6,7]]]],8,9]
[[1],4]
[[2]]
[3]
[[4,4],4,4]
[[4,4],4,4,4]
[[6]]
[7,7,7]
[7,7,7,7]
[[8,7,6]]
[9]
``````
Afterward, locate the divider packets. To find the decoder key for this distress signal, you need to determine the indices of the two divider packets and multiply them together. (The first packet is at index 1, the second packet is at index 2, and so on.) In this example, the divider packets are 10th and 14th, and so the decoder key is 140.

Organize all of the packets into the correct order. What is the decoder key for the distress signal?

In [22]:
# Loading in the input file
with open('input.txt') as f:
    distress_pairs = f.read().splitlines()

In [25]:
# Instantiating a thing to hold all our distress signals
distress_signals = []

# Adding all the relevant entries to distress signals
for entry in distress_pairs:

    # Skipping over empty lines
    if entry == '':
        continue
    else:
        distress_signals.append(ast.literal_eval(entry))

# Appending the two "break points" to this collective list
distress_signals.append([[2]])
distress_signals.append([[6]])

In [26]:
from functools import cmp_to_key

# Sorting the signals using the comparison function as a key
distress_signals = sorted(distress_signals, key = cmp_to_key(compare_items), reverse = True)

In [28]:
# Finding the break points in our newly sorted list
for i, signal in enumerate(distress_signals):

    if signal == [[2]]:
        a = i + 1
    elif signal == [[6]]:
        b = i + 1

print(f'Final result: {a * b}')

Final result: 23520
